In [1]:
import pandas as pd
import os
from collections import defaultdict
import paramiko # for ssh
import traceback #  for error handling
import ast
import requests
import re
from Bio.SeqUtils import seq1
from six.moves.urllib.request import urlopen

c:\Users\liyoa\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
cgc_data = pd.read_csv('Census_allTue Apr  9 04_58_38 2024.csv')
cgc_data.head()

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,10q11.23,yes,NaN,melanoma,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"ACF,ACF64,ACF65,APOBEC1CF,ASP,CCDS73133.1,ENSG..."
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,10p12.1,yes,NaN,AML,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"ABI-1,CCDS7150.1,E3B1,ENSG00000136754.17,NM_00..."
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,9q34.12,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL,CCDS35165.1,ENSG00000097007.17,JTK7,NM_007..."
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,1q25.2,yes,NaN,AML,NaN,NaN,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"ABLL,ARG,CCDS30947.1,ENSG00000143322.19,NM_007..."
4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,2q37.3,yes,NaN,lipoma,NaN,NaN,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"CCDS2516.1,CMKOR1,CXCR7,ENSG00000144476.5,GPR1..."


In [96]:
response = requests.get("https://gtexportal.org/api/v2/expression/geneExpression/gene-expression?gencodeId=ENSG00000065613.9,ENSG00000203782.5")
response

<Response [404]>

In [2]:
import requests

def get_pdb_ids(gene_symbol, entrez_gene_id, synonyms):
    api_url = f"http://mygene.info/v3/query?q={gene_symbol}&species=human&fields=entrezgene,ensembl,pdb"
    response = requests.get(api_url)
    # check response
    if response.status_code != 200:
        return response.status_code, None
    response = response.json()
    for hit in response['hits']:
        if ( str(hit['_id']) == str(int(entrez_gene_id)) ) or ( synonyms and 'ensembl' in hit and hit['ensembl']['gene'] in synonyms ): # (sometimes entrez_gene_id might not match but ensembl gene does
            if 'pdb' in hit:
                return 200, hit['pdb'] # status_code, pdb_ids
            else: # no structures found 
                continue
    return 200, None


In [18]:
# add pdb structures to cgc_data
pdb_structures = []
count = 1
for _, row in cgc_data.iterrows():
    # print curent count, replace previous line    
    gene_symbol = row['Gene Symbol']
    entrez_gene_id = row['Entrez GeneId']
    synonyms = row['Synonyms']
    print(f"Processing {count} of {len(cgc_data)} ({gene_symbol})", end='\r')
    count += 1

    try:
        status, pdb = get_pdb_ids(gene_symbol, entrez_gene_id, synonyms)
    except Exception as e:
        print(f"Failed to get pdb ids for {gene_symbol} with error {e}")
        pdb_structures.append(None)
        continue

    if not pdb:
        print(f"Failed to get pdb ids for {gene_symbol} with status code {status}")
        pdb_structures.append(None)
    else:
        pdb_structures.append(list(pdb))

cgc_data['PDB Structures'] = pdb_structures

Failed to get pdb ids for ACSL3 with status code 200
Failed to get pdb ids for ACSL6 with status code 200
Failed to get pdb ids for AFF3 with status code 200
Failed to get pdb ids for AKAP9 with status code 200
Failed to get pdb ids for ARHGEF10 with status code 200
Failed to get pdb ids for ARHGEF10L with status code 200
Failed to get pdb ids for ASPM with status code 200


KeyboardInterrupt: 

In [ ]:
cgc_data.head(3)

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),...,Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms,PDB Structures
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,10q11.23,yes,NaN,melanoma,...,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"ACF,ACF64,ACF65,APOBEC1CF,ASP,CCDS73133.1,ENSG...",2CPD
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,10p12.1,yes,NaN,AML,...,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"ABI-1,CCDS7150.1,E3B1,ENSG00000136754.17,NM_00...",7LXE
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,9q34.12,yes,NaN,"CML, ALL, T-ALL",...,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL,CCDS35165.1,ENSG00000097007.17,JTK7,NM_007...","[1AB2, 1AWO, 1BBZ, 1JU5, 1OPL, 1ZZP, 2ABL, 2E2..."


In [3]:
def dwl_pdb_file(pdb_id):
    base_url = 'https://files.rcsb.org/download/'
    pdb_url = base_url + pdb_id + '.pdb'
    response = requests.get(pdb_url)
    # Check if the request was successful
    if response.status_code == 200:
        # Save the PDB file to pdb_files folder
        with open(f'pdb_files/{pdb_id}.pdb', 'wb') as f:
            f.write(response.content)
    else:
        print(f"Failed to download PDB file {pdb_id}. Status code:", response.status_code)
    return

def get_resolution(pdb_id):
    try:
        with open(f'pdb_files/{pdb_id}.pdb', 'r') as f:
            pdb_text = f.read()

            # find line REMARK   2 RESOLUTION.
            resolution = None
            for line in pdb_text.split('\n'):
                if line.startswith('REMARK   2 RESOLUTION.'):
                    resolution = line.split("REMARK   2 RESOLUTION.")[1].strip()
                    break
        if "ANGSTROM" in resolution:
            resolution = resolution.replace(" ANGSTROMS.", "").strip()
        else:
            # print(f"Resolution not found for {pdb_id}")
            resolution = None
    except Exception as e:
        print(f"Failed to get resolution for {pdb_id} with error {e}")
        resolution = None
    return resolution


In [ ]:
# count number of elements in each pdb_structure
lens = []
for i in pdb_structures:
    # if value is list
    if isinstance(i, list):
        lens.append(len(i))
    elif i is not None:
        lens.append(1)
    else:
        lens.append(0)

cgc_data['n_structures'] = lens
cgc_data.head(3)

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),...,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms,PDB Structures,n_structures
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,10q11.23,yes,NaN,melanoma,...,E,NaN,oncogene,Mis,NaN,NaN,NaN,"ACF,ACF64,ACF65,APOBEC1CF,ASP,CCDS73133.1,ENSG...",2CPD,1
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,10p12.1,yes,NaN,AML,...,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"ABI-1,CCDS7150.1,E3B1,ENSG00000136754.17,NM_00...",7LXE,1
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,9q34.12,yes,NaN,"CML, ALL, T-ALL",...,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL,CCDS35165.1,ENSG00000097007.17,JTK7,NM_007...","[1AB2, 1AWO, 1BBZ, 1JU5, 1OPL, 1ZZP, 2ABL, 2E2...",81


In [ ]:
cgc_data.to_csv('Census_all_with_pdb.csv', index=False)

In [4]:
tmp_genes = ["KRAS", "EGFR", "TP53", "HRAS"]

In [19]:
# download pdb files

resolutions = defaultdict(dict)

# count number of structures for all gene
n_total = cgc_data[cgc_data['Gene Symbol'].isin(tmp_genes)]["n_structures"].sum()

for gene in tmp_genes:
    pdb_structures = cgc_data[cgc_data['Gene Symbol'] == gene]['PDB Structures'].values[0]
    if pdb_structures:
        for pdb_id in pdb_structures:
            n_total -= 1
            print(f"Downloading PDB file {pdb_id} for {gene} ({n_total} structures remaining)", end='\r')
            # check if the file already exists
            if not os.path.exists(f'pdb_files/{pdb_id}.pdb'):
                dwl_pdb_file(pdb_id)
            resolution = get_resolution(pdb_id)
            resolutions[gene][pdb_id] = resolution

In [5]:
def sort_pdb_ids_by_resolution(gene_id):
    pdb_resolutions = resolutions[gene_id]
    sorted_pdb_ids = sorted(pdb_resolutions, key=lambda pdb_id: float('inf') if pdb_resolutions[pdb_id] is None else float(pdb_resolutions[pdb_id]))
    sorted_resolutions = [(pdb_id, pdb_resolutions[pdb_id]) for pdb_id in sorted_pdb_ids]
    return sorted_resolutions

top_structures = dict()
n = 30
for gene_id in resolutions:
    sorted_pdb_resolutions = sort_pdb_ids_by_resolution(gene_id)
    print(f"\nTop {n} structures for Gene {gene_id}:")
    for pdb_id, resolution in sorted_pdb_resolutions[:n]:
        print(f"PDB ID: {pdb_id}, Resolution: {resolution}")
    top_structures[gene_id] = [i[0] for i in sorted_pdb_resolutions[:n]]

NameError: name 'resolutions' is not defined

In [6]:
top_structures = {'KRAS': ['6P0Z',
  '8ONV',
  '8AZZ',
  '4QL3',
  '8AZX',
  '8B00',
  '8AZV',
  '8AZY',
  '8B78',
  '8AFB'],
 'EGFR': ['8A27',
  '8A2D',
  '5UG9',
  '5HG8',
  '8A2A',
  '5UG8',
  '3POZ',
  '6TFV',
  '6TG0',
  '3VRP'],
 'TP53': ['3D06',
  '5MHC',
  '6GGC',
  '6SHZ',
  '4MZI',
  '6GGE',
  '3LW1',
  '5O1E',
  '6RL3',
  '8E7A'],
 'HRAS': ['2CE2',
  '2EVW',
  '2CLD',
  '2CL6',
  '2CL7',
  '5WDQ',
  '1CTQ',
  '2CLC',
  '3K8Y',
  '3OIW']}
# top_structures

In [53]:
from stat import S_ISDIR

def download_directory(sftp, remote_dir, local_dir):
    for item in sftp.listdir_attr(remote_dir):
        remote_path = remote_dir + '/' + item.filename
        local_path = os.path.join(local_dir, item.filename)
        if S_ISDIR(item.st_mode):
            os.makedirs(local_path, exist_ok=True)
            download_directory(sftp, remote_path, local_path)
        else:
            sftp.get(remote_path, local_path)

# get all chains, uniprot id, and start and end in pdb file
def get_dbref_data(pdb_id):
    with open(f'pdb_files/{pdb_id}.pdb', 'r') as f:
        pdb_text = f.read()
        data = []
        for line in pdb_text.split('\n'):
            # contents of a DBREF line:  https://www.wwpdb.org/documentation/file-format-content/format33/sect3.html
            # only reading relevant contents    
            if line.startswith('DBREF') and line[26:32].strip()=="UNP": # if database is UNP 
                data.append({
                    'chain': line[12],
                    'uniprot': line[33:41].strip(),
                    'start': int(line[14:18]),
                    'end': int(line[20:24])
                })  
    return data

get_dbref_data('2F1X')

[{'chain': 'A', 'uniprot': 'Q93009', 'start': 53, 'end': 200},
 {'chain': 'A', 'uniprot': 'P04637', 'start': 201, 'end': 209},
 {'chain': 'B', 'uniprot': 'Q93009', 'start': 53, 'end': 200},
 {'chain': 'B', 'uniprot': 'P04637', 'start': 201, 'end': 209}]

In [ ]:
def parse_dbref_line(line):
    # contents of a DBREF line:  https://www.wwpdb.org/documentation/file-format-content/format33/sect3.html
    # only reading relevant contents
    chain_id = line[12]               # Chain identifier
    seq_begin = int(line[14:18])      # Initial sequence number of the PDB sequence segment
    seq_end = int(line[20:24])        # Ending sequence number of the PDB sequence segment
    database = line[26:32].strip()    # Sequence database name
    db_accession = line[33:41].strip()# Sequence database accession code
    
    return {
        "chain_id": chain_id,
        "seq_begin": seq_begin,
        "seq_end": seq_end,
        "database": database,
        "db_accession": db_accession,
    }

# Example usage:
line = "DBREF  6LHD A    2   149L UNP    Q07817   B2CL1_HUMAN      2    201"
parsed_line = parse_dbref_line(line)
print(parsed_line)

In [8]:
def calculate_sbna_and_download(client, sftp, pdb_id, base_path, chains=None):
    # get available chains
    if not chains: # chain not specified
        try:
            chains = get_dbref_data(pdb_id)
            chains = [chain['chain'] for chain in chains]
        except Exception as e:
            print(f"Failed to get chains for {pdb_id} with error {e}")
            return
    
    sftp.mkdir(f'{base_path}/{pdb_id}')
    for chain in chains: 
        local_dir = f"sbna_results/{pdb_id}/{chain}"
        if os.path.exists(f"{local_dir}/{pdb_id}_monomer/FinalSum"): # skip if analysis already done and downloaded
            continue
        try:
            print(f"\nProcessing {pdb_id} with chain {chain}...")

            # remove the directory if already exists and create new one
            client.exec_command(f"rm -r {base_path}/{pdb_id}/{pdb_id}_monomer")
            client.exec_command(f"rm -r {base_path}/{pdb_id}/{pdb_id}_multimer") 
            
            # upload pdb file to the remote directory
            sftp.put(f"pdb_files/{pdb_id}.pdb", f'{base_path}/{pdb_id}/{pdb_id}.pdb') 
            
            # copy the sbna.sh file to the subdirectory
            client.exec_command(f"cp {base_path}/sbna.sh {base_path}/{pdb_id}/sbna.sh") 

            # run the sbna.sh script
            _,stdout,stderr=client.exec_command(f'cd {base_path}/{pdb_id}; sh sbna.sh {pdb_id}.pdb "Chain {chain}"') 
            print(stdout.read().decode())
            
            # check if Final_sum file exists
            print(stderr.read().decode('utf-8'))
            _,stdout,stderr=client.exec_command(f"ls {base_path}/{pdb_id}/{pdb_id}_monomer/FinalSum")
            # print error message if no output

            # download the sbna results
            if not stdout.read().decode():
                print(f"Final_sum file not found for {pdb_id}")
                # client.exec_command(f"rm -rf {base_path}/{pdb_id}") 
                continue
            
            # download the pdb_id directory
            os.makedirs(local_dir, exist_ok=True) # create the directory if it doesn't exist
            download_directory(sftp, f"{base_path}/{pdb_id}", local_dir)
            print(f"SBNA results for {pdb_id} succesfully downloaded")
        
        except Exception:
            # print full error message
            print(f"Failed to process {pdb_id} with error:")
            print(traceback.print_exc())

            # remove directory
            client.exec_command(f"rm -r {base_path}/{pdb_id}/{pdb_id}_monomer")
            client.exec_command(f"rm -r {base_path}/{pdb_id}/{pdb_id}_multimer")
            continue
    
    client.exec_command(f"rm -rf {base_path}/{pdb_id}") # remove the pdb_id remote directory
    return
    

In [102]:
host = "m3-dtn.massive.org.au"
username = "yliy0004"
with open("pw.txt", "r") as f:
    password = f.read()
base_path = 'ym65_scratch/yliy0004/NetworkAnalysis'

client = paramiko.client.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(host, username=username, password=password)

# check if R module is loaded (uses .bashrc - not the best solution because STRUDEL cannot be used, make sure to correct this if using STRUDEL)
stdin,stdout,stderr=client.exec_command('R --version') 
print(stdout.read().decode())
print(stderr.read().decode())

sftp_client=client.open_sftp()

# ==================== SBNA ====================
# multiple structures by gene
# for gene_id in top_structures:
    # for pdb_id in top_structures[gene_id]:

# or certain genes
# for gene in ['TP53', 'HRAS', 'KRAS', 'EGFR']:
#     count = 0
#     tmp = ast.literal_eval(cgc_data.loc[cgc_data['Gene Symbol'] == gene, 'PDB Structures'].values[0])
#     for pdb_id in tmp:
#         try:
#             resolution = get_resolution(pdb_id)
#             if resolution and float(resolution) <= 3.0:
#                 calculate_sbna_and_download(client, sftp_client, pdb_id, base_path)
#         except Exception as e:
#             print(f"Failed to process {pdb_id} with error:")
#             print(traceback.print_exc())
#             continue

# or one pdb
calculate_sbna_and_download(client, sftp_client, "1TSR", base_path, chains=['C'])

# close the connection
client.close()
sftp_client.close()

R version 4.0.5 (2021-03-31) -- "Shake and Throw"
Copyright (C) 2021 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under the terms of the
GNU General Public License versions 2 or 3.
For more information about these matters see
https://www.gnu.org/licenses/.




Processing 1TSR with chain C...
Starting phenix.pdbtools
on Thu Apr 18 13:23:24 2024 by yliy0004

Processing files:
-------------------------------------------------------------------------------

  Found model, 1TSR.pdb

Processing PHIL parameters:
-------------------------------------------------------------------------------

  Adding command-line PHIL:
  -------------------------
    keep=Chain C

Final processed PHIL parameters:
-------------------------------------------------------------------------------
  data_manager {
    model {
      file = "1TSR.pdb"
    }
    default_model = "1TSR.pdb"


In [9]:
client.close()
sftp_client.close()

In [67]:
cgc_data = pd.read_csv('Census_all_with_pdb.csv')

In [16]:
def get_uniprot_id(pdb_id, chain):
    # read pdb file 
    with open(f'pdb_files/{pdb_id}.pdb', 'r') as f:
        pdb_text = f.read()
        for line in pdb_text.split('\n'):
            vals = line.split()
            if line.startswith('DBREF') and chain in vals[2] and 'UNP' in vals:
                return vals[6] # uniprot id is the 6th value


def fetch_sequence(uniprot_id):
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"
    response = requests.get(url)
    if response.ok:
        # Extract the sequence from the FASTA format
        lines = response.text.split('\n')
        sequence = ''.join(lines[1:])  # Skip the header line
        return sequence
    else:
        print("Failed to fetch sequence")
        return None


def get_gene_name(uniprot_id):
    response = requests.get(f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta")
    response = response.text
    match = re.search(r'GN=([^\s]+)', response)
    if match:
        return match.group(1)
    else:
        return None
    
    
def read_finalsum(pdb_id, chain):
    if os.path.exists(f'sbna_results/{pdb_id}/{chain}/{pdb_id}_monomer/FinalSum'):
        with open(f'sbna_results/{pdb_id}/{chain}/{pdb_id}_monomer/FinalSum', 'r') as f:
            # convert final_sum to dataframe
            final_sum = f.readlines()
            final_sum = [i.split() for i in final_sum]
            rows = []
            for row in final_sum:
                # get letters only from row[0]
                aa_3 = ''.join(filter(str.isalpha, row[0]))
                if row[1] == 'NA' or len(aa_3) != 3: 
                    # skip if residue number is NA or residue code is not 3 letters
                    continue
                
                aa_1 = seq1(aa_3) # convert to 1 letter code
                res_num = ''.join(filter(str.isnumeric, row[0]))
                score = float(row[1])
                # print(f"{aa_3} {aa_1} {res_num} {score}")
                rows.append([aa_3, aa_1, int(res_num), score])
            final_sum = pd.DataFrame(rows, columns=['res', 'res_code', 'num', 'score'])
        return final_sum
    else:
        print(f"FinalSum file not found for {pdb_id}")
        return None

In [86]:
# tmp_genes = ["KRAS", "EGFR", "TP53", "HRAS"]
# tmp_genes = ["KRAS", "TP53", "HRAS"]
tmp_genes = ["EGFR"]

In [ ]:
final_data = []

In [87]:
final_data = list(final_data.values)

In [88]:
filtered_data = cgc_data[cgc_data['PDB Structures'].notna()] # pdb_structures not nan
filtered_data = filtered_data[filtered_data['Gene Symbol'].isin(tmp_genes)]

for i, row in filtered_data.iterrows():
    gene_symbol = row['Gene Symbol']
    pdb_structures = row['PDB Structures']
    for pdb_id in ast.literal_eval(pdb_structures):
        if not os.path.exists(f'pdb_files/{pdb_id}.pdb'):
            continue
        else:
            data = get_dbref_data(pdb_id)
            resolution = get_resolution(pdb_id)
            for i in data:
                chain = i['chain']

                # check if finalsum exists or if data is already in final_data
                if not os.path.exists(f'sbna_results/{pdb_id}/{chain}/{pdb_id}_monomer/FinalSum') or any([pdb_id == i[1] and chain == i[2] for i in final_data]):
                    continue    
                      
                uniprot_id = i['uniprot']
                assoc_gene = get_gene_name(uniprot_id)
                if assoc_gene != gene_symbol:
                    # sometimes protein chain extends multiple genes, skip if not the same
                    # in future can link if gene is in cgc_data/tcga_data
                    continue

                # read finalSum file and add to table
                final_sum = read_finalsum(pdb_id, chain)

                # check if seqeunce of final sum matches uniprot
                uniprot_seq = fetch_sequence(uniprot_id)
                
                for _, row in final_sum.iterrows():
                    num = row['num']
                    start = i['start']
                    end = i['end']
                    outside_range = False
                    residue_match = False
                    try:
                        # get residue from uniprot sequence
                        uniprot_res = uniprot_seq[num-1]
                    except:
                        uniprot_res = None

                    if num < start or num > end: # check if residue number is within range
                        outside_range = True
                        # print(f"Residue {num} not in range {start} - {end}")
                    if row['res_code'] == uniprot_res: # check if residue code matches
                        residue_match = True

                    final_data.append([gene_symbol, pdb_id, chain, uniprot_id, assoc_gene, resolution, num, row['res_code'], uniprot_res, row['score'], outside_range, residue_match])


In [89]:
final_data = pd.DataFrame(final_data, columns=['gene_symbol', 'pdb_id', 'chain', 'uniprot_id', 'assoc_gene', 'resolution', 'res_num', 'pdb_res', 'uniprot_res', 'network_score', 'outside_range', 'residue_match'])
final_data

,gene_symbol,pdb_id,chain,uniprot_id,assoc_gene,resolution,res_num,pdb_res,uniprot_res,score,outside_range,residue_match
0,TP53,1C26,A,P04637,TP53,1.70,325,G,G,-0.148266,False,True
1,TP53,1C26,A,P04637,TP53,1.70,326,E,E,-0.982754,False,True
2,TP53,1C26,A,P04637,TP53,1.70,327,Y,Y,0.164008,False,True
3,TP53,1C26,A,P04637,TP53,1.70,328,F,F,-0.577961,False,True
4,TP53,1C26,A,P04637,TP53,1.70,329,T,T,-0.808752,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
28815,EGFR,8A2D,A,P00533,EGFR,1.11,1017,L,L,-0.876917,False,True
28816,EGFR,8A2D,A,P00533,EGFR,1.11,1018,I,I,-1.457035,False,True
28817,EGFR,8A2D,A,P00533,EGFR,1.11,1019,P,P,-3.161216,False,True
28818,EGFR,8A2D,A,P00533,EGFR,1.11,1020,Q,Q,-2.024879,False,True


In [90]:
final_data.to_csv('final_data.csv', index=False)